In [1]:
"""
This is the script we use to generate our schedulers clean accuracy, adversarial accuracy, runtime and others.
we averaged multiple runs in our repport
"""

'\nThis is the script we use to generate our schedulers clean accuracy, adversarial accuracy, runtime and others.\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
#sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '../../..')))

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import model.Models as Models
import Defences as Defences
import schedulers.Schedulers as Schedulers
import pandas as pd
import numpy as np
from Attacks import pgd_attack
import torch.nn.functional as F

In [4]:
"""
Hyper parameters, feel free to experiment.
The commented ones are the ones we used on CIFAR10
"""

# MNIST SetUp ------------------------
device = "cuda"

dataset = "MNIST" 
batchsize = 64 

k_max = 20 
epsilon = 0.3 
epochs = 8 
lr = 1e-3

def get_model():
    return Models.MediumConvNet().to(device) # See our Models.file, we also implemented ResNets and such

def get_optimizer(model):
    return torch.optim.Adam(model.parameters(), lr=lr)

# CIFAR SetUp ------------------------
"""
dataset = "CIFAR10"
batchsize = 200

k_max = 7
epsilon = 255/8
epochs = 20
lr = 1e-4

def get_model():
    return Models.MediumConvNet().to(device) # See our Models.file, we also implemented ResNets and such
lr = 1e-4

def get_optimizer():
    return torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
"""

'\ndataset = "CIFAR10"\nbatchsize = 200\n\nk_max = 7\nepsilon = 255/8\nepochs = 20\nlr = 1e-4\n\ndef get_model():\n    return Models.MediumConvNet().to(device) # See our Models.file, we also implemented ResNets and such\nlr = 1e-4\n\ndef get_optimizer():\n    return torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)\n'

In [5]:
def get_data_loaders(batch_size=64, dataset = dataset):
    if dataset == "MNIST":
        print("Loading MNIST data...")
        transform = transforms.ToTensor()
        train_dataset = datasets.MNIST(root="../../project_code/data", train=True, download=True, transform=transform)
        test_dataset = datasets.MNIST(root="../../project_code/data", train=False, download=True, transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)
        print("MNIST data loaded.")
        return train_loader, test_loader

    elif dataset == "CIFAR10":
        """Get CIFAR-10 data loaders with appropriate transforms"""
        # CIFAR-10 specific transforms
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])
        
        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])
        
        train_dataset = datasets.CIFAR10(root="../data", train=True, download=True, transform=transform_train)
        test_dataset = datasets.CIFAR10(root="../data", train=False, download=True, transform=transform_test)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
        test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False, num_workers=2)
        
        return train_loader, test_loader
    

In [6]:
def evaluate_model(model, test_loader, criterion, device):
    """
    Evaluate model on clean examples
    """
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    test_loss /= test_total
    test_acc = 100.0 * test_correct / test_total
    return test_loss, test_acc

In [7]:
def run_all_k_strategies(k_min=0, k_max=k_max, epsilon=epsilon, num_epochs=epochs, device=device, lr = lr):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    print(f"Training on {device}")
    
    train_loader, test_loader = get_data_loaders()
    results = []
    schedulers = {
        "Vanilla": Schedulers.VanillaScheduler(),
        "Constant": Schedulers.ConstantScheduler(k_min, k_max),
        "LinearUniformMix": Schedulers.LinearUniformMixScheduler(k_min, k_max),
        "Exponential": Schedulers.ExponentialScheduler(k_min, k_max),
        "Cyclic": Schedulers.CyclicScheduler(k_min, k_max),
        "Random": Schedulers.RandomScheduler(k_min, k_max),
        "Constant": Schedulers.ConstantScheduler(k_min, k_max),
    }
    
    os.makedirs("results", exist_ok=True)
    for name, scheduler in schedulers.items():
        print(f"\nTraining with {name} scheduler...")
        
        #model = Models.MediumConvNet().to(device)  # Using larger model
        #optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        model = get_model()
        optimizer = get_optimizer(model)
        
        criterion = nn.CrossEntropyLoss()
        Defences.train_with_adversarial_scheduler(
            model, train_loader, test_loader, optimizer, criterion,
            epsilon, scheduler, device, num_epochs=num_epochs, test_eval_rate=2
        )
        test_loss, test_acc = evaluate_model(model, test_loader, criterion, device)
        results.append({
            "strategy": name,
            "test_loss": test_loss,
            "test_accuracy": test_acc
        })
        # Save model after training
        torch.save(model.state_dict(), f"results/model_{name}.pth")
    # Save results to CSV
    pd.DataFrame(results).to_csv("results/k_strategy_results.csv", index=False)
    print("\nSummary of results:")
    print(pd.DataFrame(results))

In [8]:
def load_models(strategies, device):
    model_dict = {}
    for name in strategies:
        model = get_model()  
        model.load_state_dict(torch.load(f"results/model_{name}.pth", map_location=device))
        model.eval()
        model_dict[name] = model
    return model_dict


def evaluate_strategies_on_attacks(model_dict, test_loader, device, epsilon=epsilon, k_list=[1,2,4,8,16]):
    results = []
    criterion = nn.CrossEntropyLoss()
    for strategy, model in model_dict.items():
        # Clean accuracy
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        clean_acc = 100.0 * correct / total

        # Adversarial accuracy for each k
        for k in k_list:
            correct_adv = 0
            total_adv = 0
            confidences = []
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                adv_images, _ = pgd_attack(images, labels, model, criterion, epsilon, k, device)
                outputs = model(adv_images)
                probs = F.softmax(outputs, dim=1)
                _, predicted = outputs.max(1)
                total_adv += labels.size(0)
                correct_adv += (predicted == labels).sum().item()
                # Mean confidence on correct predictions
                for i in range(labels.size(0)):
                    confidences.append(probs[i, predicted[i]].item())
            adv_acc = 100.0 * correct_adv / total_adv
            mean_conf = float(np.mean(confidences)) if confidences else 0.0
            results.append({
                "strategy": strategy,
                "k": k,
                "clean_acc": clean_acc,
                "adv_acc": adv_acc,
                "mean_confidence": mean_conf
            })
    df = pd.DataFrame(results)
    df.to_csv("results/adversarial_evaluation.csv", index=False)
    print("\nAdversarial evaluation results:")
    print(df)
    return df

In [9]:
"""
Generate results
"""
run_all_k_strategies()

Training on cuda
Loading MNIST data...
MNIST data loaded.

Training with Vanilla scheduler...
Epoch 1/8 | Train Loss: 0.2780, Train Acc: 91.28%
Epoch 2/8 | Train Loss: 0.0875, Train Acc: 97.48% | Test Loss: 0.0351, Test Acc: 98.87%
Test Adv Loss (k=7): 7.6994, Test Adv Acc: 0.58%
Epoch 3/8 | Train Loss: 0.0597, Train Acc: 98.30%
Epoch 4/8 | Train Loss: 0.0495, Train Acc: 98.56% | Test Loss: 0.0387, Test Acc: 98.65%
Test Adv Loss (k=7): 10.4333, Test Adv Acc: 0.56%
Epoch 5/8 | Train Loss: 0.0439, Train Acc: 98.73%
Epoch 6/8 | Train Loss: 0.0349, Train Acc: 98.99% | Test Loss: 0.0252, Test Acc: 99.26%
Test Adv Loss (k=7): 12.2586, Test Adv Acc: 0.01%
Epoch 7/8 | Train Loss: 0.0307, Train Acc: 99.08%
Epoch 8/8 | Train Loss: 0.0263, Train Acc: 99.21% | Test Loss: 0.0237, Test Acc: 99.24%
Test Adv Loss (k=7): 14.3857, Test Adv Acc: 0.09%
Training time (without eval): 1 min 0.05 sec

Training with Constant scheduler...
Epoch 1/8 | Train Loss: 2.3022, Train Acc: 11.01%
Epoch 2/8 | Train Loss:

In [10]:
# Evaluation phase

_, test_loader = get_data_loaders()
strategies = ["Vanilla", "Constant", "Linear", "LinearUniformMix", "Exponential", "Cyclic", "Random"]
model_dict = load_models(strategies, device)
evaluate_strategies_on_attacks(model_dict, test_loader, device)

Loading MNIST data...
MNIST data loaded.

Adversarial evaluation results:
            strategy   k  clean_acc  adv_acc  mean_confidence
0            Vanilla   1      99.24    15.66         0.714379
1            Vanilla   2      99.24     2.61         0.863207
2            Vanilla   4      99.24     0.28         0.951862
3            Vanilla   8      99.24     0.04         0.978362
4            Vanilla  16      99.24     0.00         0.984970
5           Constant   1      11.35    11.35         0.113462
6           Constant   2      11.35    11.35         0.113462
7           Constant   4      11.35    11.35         0.113462
8           Constant   8      11.35    11.35         0.113462
9           Constant  16      11.35    11.35         0.113462
10            Linear   1      98.81    96.17         0.966872
11            Linear   2      98.81    95.62         0.964467
12            Linear   4      98.81    95.31         0.962138
13            Linear   8      98.81    95.00         0.961

,strategy,k,clean_acc,adv_acc,mean_confidence
0,Vanilla,1,99.24,15.66,0.714379
1,Vanilla,2,99.24,2.61,0.863207
2,Vanilla,4,99.24,0.28,0.951862
3,Vanilla,8,99.24,0.04,0.978362
4,Vanilla,16,99.24,0.00,0.984970
5,Constant,1,11.35,11.35,0.113462
6,Constant,2,11.35,11.35,0.113462
7,Constant,4,11.35,11.35,0.113462
8,Constant,8,11.35,11.35,0.113462
9,Constant,16,11.35,11.35,0.113462
